# MOULIN'S WATER LEVEL TIMESCALE FLUCTUATIONS


    1.Reservoir-constriction equation from eq(5a)&(5b)
    2.Creep equation from eq(6) in Covington(2012)-> derived from Spring&Hutter(1981), 
    and also used in Arnold et al.(1998) model
    
![title](SketchMoulin.png)

    Constants:
    g = 9.81 m/s2
    
    Fixed variables:
    R 
    
    

In [21]:
%pylab
from scipy.integrate import odeint
from scipy import signal
import matplotlib.pyplot as plt
#import numpy as np

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [22]:
# RESERVOIR-CONSTRICTION
########################

# Constant:
g = 9.81 #m/s2
Lf = 3.34*10.**5 # J/kg
n = 3. # Unitless, ice flow law exponent
B = 5.8*10.**7 # N/m^2*s, Arrhenius parameter
rhow = 1000. # kg/m^3, density of water
rhoi = 900. # kg/m^3, density of ice
f = 0.1 #unitless


def dh_Ac_dt(parameter, time, Recharge, Thickness, M_area, C_length):
    Fh,FAc = parameter
    D = sqrt(FAc*4./pi)
    #FAc = pi*(D**2)/4.
    Cf = 1. + f*Thickness/D
    Pwet = 2.*pi*D/2 # conduit wetted perimeter
    Q = FAc*sqrt(2.*g*Fh/Cf)
    Pw = rhow*g*Fh
    Pi = rhoi*g*Thickness    
    melt = (f*rhow*Pwet*Q**3.)/(8.*rhoi*Lf*FAc**3.)
    creep = 2.*(1./(n*B)**n)*FAc*(Pi-Pw)*abs(Pi-Pw)**(n-1.)
    return (Recharge-Q)/M_area, melt-creep





In [23]:
# odeint simple
R = array([1., 3., 6.]) #m^3/s
Z = 1000. # m, thickness of the ice
r = 5. # m, radius of the moulin
Ar = pi*r**2. # m^2
L = 1000. #m
Days = 10     ; duration = Days*24*3600; tstep = duration/100; time = linspace(0,duration,tstep)
initialpara=[200.,0.1]
t = time/(24*3600) # from seconds to days


result = odeint(dh_Ac_dt, initialpara, time, args=(R[0], Z, Ar, L))
h1 = result[:,0]
Ac1 = result[:,1]
print h1

[ 200.          200.80018689  201.60520633 ...,  298.37254801  298.21091209
  298.05417485]


In [27]:
# Odeint with a loop of parameters
R = array([1.,2.,3.,4.,5.,6.,7.,8.,9.,10.]) #m^3/s
Z = array([100.,200.,300.,400.,500.,600.,700.,800.,900.,1000.]) # m, thickness of the ice
r = array([1.,2.,3.,4.,5.,6.,7.,8.,9.,10.]) # m, radius of the moulin
L = 1000. #m
Days = 1000.     ; duration = Days*24.*3600.; tstep = duration/100.; time = linspace(0,duration,tstep)
initialpara=[200.,0.1]
t = time/(24*3600) # from seconds to days
tsc = np.zeros((10,10,10))
for i in [0,1,2,3,4,5,6,7,8]:
    for j in [0,1,2,3,4,5,6,7,8]:
        for k in [0,1,2,3,4,5,6,7,8]:
            #h=[]
            Ar = pi*r[k]**2. # m^2
            result = odeint(dh_Ac_dt, initialpara, time, args=(R[j], Z[i], Ar, L))
            h = result[:,0]
            Ac = result[:,1]
            f, Pxx_den = signal.periodogram(h)
            pic = f[Pxx_den.argmax()]
            tsc[i][j][k]=1/pic*100/3600


RuntimeError: The array return by func must be one-dimensional, but got ndim=2.

In [26]:
h

[]

In [ ]:
figure()
f, Pxx_den = signal.periodogram(h)
plt.semilogy(f, Pxx_den)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

#figure(figsize=(15,4))
figure()
plot(t,h); title('Hydraulic head in the moulin'); ylabel('[m]'); xlabel('Day'); grid()
#figure(figsize=(15,4))
figure()
plot(t,Ac); title('Conduit Diameter'); ylabel('[m]'); xlabel('Day'); grid()


In [ ]:
pic = f[Pxx_den.argmax()]  # Find the x value corresponding to the maximum y value
print pic
timescale=1/pic*100/3600
print timescale

In [ ]:
figure(figsize=(15,4))
plot(t,h); title('Hydraulic head in the moulin'); ylabel('[m]'); xlabel('Day'); grid()

savefig('Moulin_HydraulicHead.pdf')
figure(figsize=(15,4))
plot(t,Dc); title('Conduit Diameter'); ylabel('[m]'); xlabel('Day'); grid()
savefig('Moulin_ConduitDiameter.pdf')